# 1.清单计算

In [10]:
import csv,xlrd,xlwt,datetime,time,re
import numpy as np

import pandas as pd




#一、导入指导价

#二、打开对方清单
def suanqingdan():
    suanqingdan=0
    global url,qing,qing2,shijian0
    jia_df=pd.read_excel('转贴现买断价格计算/jia.xls',encoding='GB18030')

    shijian0=time.strftime('%Y-%m-%d',time.localtime(time.time()))
    shijian=datetime.datetime.strptime(shijian0 , "%Y-%m-%d")#转为日期格式

#2.1输入交易日
    jin=input('请输入要交易日,输入格式为“2017-11-01”，不输入默认为今天:')
    if jin!='':
        shijian=datetime.datetime.strptime(jin , "%Y-%m-%d")
    jia_df['生效截至日'] = pd.to_datetime(jia_df['生效截至日'])#.astype('str')   #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式 注意要换成string格式
    jia_df=jia_df[((jia_df['生效截至日']>=shijian) | (jia_df['生效截至日']==''))&(jia_df['银行分类']!='资管、证券、金融公司')]

    jia_df= jia_df.reset_index(drop=True)    #重新定义索引
    qixian1=jia_df.columns[2].split('至')
    qixian2=jia_df.columns[3].split('至')
    qixian3=jia_df.columns[4].split('至')
    qixian4=jia_df.columns[5].split('至')
    qixian5=jia_df.columns[6].split('至')

    qixian1[0] = datetime.datetime.strptime(qixian1[0] , "%Y-%m-%d")
    qixian1[1] = datetime.datetime.strptime(qixian1[1] , "%Y-%m-%d")
    qixian2[0] = datetime.datetime.strptime(qixian2[0] , "%Y-%m-%d")
    qixian2[1] = datetime.datetime.strptime(qixian2[1] , "%Y-%m-%d")
    qixian3[0] = datetime.datetime.strptime(qixian3[0] , "%Y-%m-%d")
    qixian3[1] = datetime.datetime.strptime(qixian3[1] , "%Y-%m-%d")
    qixian4[0] = datetime.datetime.strptime(qixian4[0] , "%Y-%m-%d")
    qixian4[1] = datetime.datetime.strptime(qixian4[1] , "%Y-%m-%d")
    qixian5[0] = datetime.datetime.strptime(qixian5[0] , "%Y-%m-%d")
    qixian5[1] = datetime.datetime.strptime(qixian5[1] , "%Y-%m-%d")


#2.2确认列名，到时候可以删除
    qing=pd.read_excel(url)
    print(qing.head())
#二、换列标题
    qingcolumn=qing.columns.values.tolist()  
    qingcolumn_df=pd.read_excel('转贴现买断价格计算/lieming.csv',encoding='GB18030')
    for i in range(0,len(qingcolumn)):
        for j in range(0,len(qingcolumn_df)):
            zhaop2= re.search(qingcolumn_df.loc[j,'原列名'] ,qingcolumn[i])
 

            if zhaop2:
                qing.rename(columns={qingcolumn[i]:qingcolumn_df.loc[j,'新列名']}, inplace=True)
                break

    if '贴现行' not in qing.columns.values.tolist():
        qing['贴现行']=''


#三、筛选列，计算清单
    qing=qing[['票号','票面金额','出票人','承兑行','贴现行','出票日','到期日']]
    qing= qing.reset_index(drop=True)    #重新定义索引
    for i in range(0,len(qing)):
        qing.loc[i,'承兑行号']=str(qing.loc[i,'票号'])[1:13]
        
        
    qing['到期日2'] = pd.to_datetime(qing['到期日'])   #把str转化为时间格式，format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式
    
    qing= qing.reset_index(drop=True)    #重新定义索引
    qing['承兑行号']=qing['承兑行号'].astype(str)
    
  #  qing['出票日']=qing['出票日'].astype(str)
    qing['出票日'] = pd.to_datetime(qing['出票日']).astype(str)   #把str转化为时间格式，format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式    
    qing['到期日'] = pd.to_datetime(qing['到期日']).astype(str)   #把str转化为时间格式，format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式    
        
  #  qing['到期日']=qing['到期日'].astype(str)

    for i in range (0,len(qing)):     
         if (qing.loc[i,'到期日2']>=qixian1[0]) and (qing.loc[i,'到期日2']<=qixian1[1]) :
           for j in range (0,len(jia_df)):
               if (jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']) and (float(jia_df.loc[j,jia_df.columns[2]])>=1) :
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[2]]+0.15
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[2]]
                       qing.loc[i,'小票票据']=0
                    

                   qing.loc[i,'指导期限']=jia_df.columns[2]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
               
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')

                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian2[0]) and (qing.loc[i,'到期日2']<=qixian2[1]) :

           for j in range (0,len(jia_df)):

               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']    and (float(jia_df.loc[j,jia_df.columns[3]])>=1) :
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[3]]+0.15
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[3]]
                       qing.loc[i,'小票票据']=0
                    
                   qing.loc[i,'指导期限']=jia_df.columns[3]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]
                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian3[0]) and (qing.loc[i,'到期日2']<=qixian3[1])  :  #

           for j in range (0,len(jia_df)):
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']  and (float(jia_df.loc[j,jia_df.columns[4]])>=1):
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[4]]+0.15
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[4]]
                       qing.loc[i,'小票票据']=0

                   qing.loc[i,'指导期限']=jia_df.columns[4]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian4[0]) and (qing.loc[i,'到期日2']<=qixian4[1])  :

           for j in range (0,len(jia_df)) :
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行'] and (float(jia_df.loc[j,jia_df.columns[5]])>=1):
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[5]]+0.15
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[5]]
                       qing.loc[i,'小票票据']=0
                   qing.loc[i,'指导期限']=jia_df.columns[5]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1




         elif (qing.loc[i,'到期日2']>=qixian5[0]) and (qing.loc[i,'到期日2']<=qixian5[1])  :

           for j in range (0,len(jia_df)) :
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行'] and (float(jia_df.loc[j,jia_df.columns[6]])>=1):
                   if qing.loc[i,'票面金额']<1000000:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[6]]+0.15
                       qing.loc[i,'小票票据']=1
                   else:
                       qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[6]]
                       qing.loc[i,'小票票据']=0
                   qing.loc[i,'指导期限']=jia_df.columns[6]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1
                   
    if suanqingdan<=0:
        print('整个清单都不能收！或者请清单检查格式。提示：1.出票日、到期日要是日期格式，2.是否有其他“承兑”、“付款”之类的列名存在，系统误认为是错误的“承兑行”列。')
        del qing['到期日2']
        qing2=qing
        
    else:
        qing['加权利率']=qing.loc[:,'ji2'].sum()/qing.loc[:,'ji1'].sum()
        del qing['到期日2']



        qing=qing[['票号','票面金额','出票人','承兑行','承兑行号','出票日','到期日','交易日','银行分类','邮储指导价','小票票据','指导期限','加权利率','贴现行','可用额度','我行评级','生效截至日','授信有效','期限','到期月']]
#三、将算好的清单写入excel
        qing2=qing.replace(np.nan,99)
     
        qing2=qing2[qing2['邮储指导价'] != 99]  #排除NAN
        chengdui=qing2['票面金额'].groupby(qing2['银行分类']).sum()/10000
        print('  ')

        zongjin=qing2['票面金额'].sum()/10000
        print('总金额：%s'%zongjin)
        print('  清单分类如下：')

        print(chengdui)
        print('  ')
        yuefen=qing2['票面金额'].groupby(qing2['到期月']).sum()/10000
        print(yuefen)
        print('  ')
        qixian=qing2['期限'].mean()
        print(qixian)
        print('  ')
        print('提示：票据持有方式选：持有！申报清单总金额和张数是否和对方实际发来的一致？承兑行是否有错别字？')

    
    





#四、制作导
        qing2['导汇票']=1
        qing2['导出票人账号']=''
        qing2['导收款人名称']=''
        qing2['导1']=''
        qing2['导2']=''
        qing2['导3']=''
        qing2['同城']=1
        qing2=qing2[['导汇票','票号','票面金额','出票日','到期日','出票人','导出票人账号','承兑行号','承兑行','导收款人名称','导1','导2','导3','承兑行号','同城','交易日','期限']]

        qing2=qing2.fillna('')  #把日期列的nan 数值用空值填充，统一空值

        print(qing2)
        qing2=qing2[(qing2['票号'] !='')]
        qing2= qing2.reset_index(drop=True)    #重新定义索引
        qing=qing.fillna('')  #把日期列的nan 数值用“NAN”填充，统一空值


        qing=qing[(qing['票号'] !='')]
        qing= qing.reset_index(drop=True)    #重新定义索引
        
        
def heimingdan():
    global qing,qing2,shijian0
    hei1=pd.read_excel('转贴现买断价格计算/系统黑名单.xls',encoding='GB18030')
  #  hei2=pd.read_excel('转贴现买断价格计算/系统黑名单企业.xls',encoding='GB18030')           
    
    for i in range(0,len(hei1)):
        if '/' in  hei1.loc[i,'银行黑名单']:      
            erhei= hei1.loc[i,'银行黑名单'].split('/')
            for j in range(0,len(qing)):
                if (erhei[0] in qing.loc[j,'承兑行']) and (erhei[1] in qing.loc[j,'承兑行']):
                    qing.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
                    qing2.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
            
                   # break
            
            
            
        else:
            for j in range(0,len(qing)):
                if hei1.loc[i,'银行黑名单'] in qing.loc[j,'承兑行']:
                    qing.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
                    qing2.loc[j,'黑名单']=hei1.loc[i,'黑名单类型']
                  #  break
    url3='临时清单/'+shijian0+duifang+'导入.xls'
    writer = pd.ExcelWriter(url3)
    qing2.to_excel(writer,'Sheet1')
    writer.save()

    url2='临时清单/'+shijian0+duifang+'算加权.xls'
    writer = pd.ExcelWriter(url2)
    qing.to_excel(writer,'Sheet1')
    writer.save()

        
for i in range(0,1):
    print('\n')
    duifang=input('请输入第%s个文件名：'%i)
 #   duifang=''+duifang
    url=('临时清单/%s'%duifang)
    if duifang=='':
        continue
    else:
        
        suanqingdan()
        heimingdan() 




请输入第0个文件名：8.23票据.xlsx
请输入要交易日,输入格式为“2017-11-01”，不输入默认为今天:
                              票据号      票据金额        出票日        到期日  \
0  131333209003520190301355438859  200000.0 2019-03-01 2019-09-01   
1  131345861606320190301355385821  100000.0 2019-03-01 2019-09-01   
2  132229000808920190305356513067  200000.0 2019-03-05 2019-09-05   
3  131331409999020190307358217569   68000.0 2019-03-07 2019-09-06   
4  131334501020520190301355745780  200000.0 2019-03-01 2019-09-01   

            出票人                 承兑行  
0    宁波曼捷贸易有限公司        宁波通商银行股份有限公司  
1   高密市宇瑞纺织有限公司      潍坊银行股份有限公司高密支行  
2  达亚帆布（上海）有限公司  上海农村商业银行股份有限公司金山支行  
3  江苏文光车辆附件有限公司      江苏银行股份有限公司镇江分行  
4   台州市博森模具有限公司      浙江泰隆商业银行台州黄岩支行  
  
总金额：41840.459917
  清单分类如下：
银行分类
一类准入                 43.000000
二类准入、外资            3748.180000
其他授信银行             3893.900259
商票、授信财务公司（国股贴）       30.977586
国股（授信银行贴）         29050.088381
大商（授信银行贴）          5074.313691
Name: 票面金额, dtype: float64
  
到期月
2019-10     2835.900685
2019-11     3305.6

PermissionError: [Errno 13] Permission denied: '临时清单/2019-08-268.23票据.xlsx算加权.xls'

# 重要交易对手同业存单

In [1]:
# -*- coding: cp936 -*-
import datetime
import json,csv,re,time
import pandas as pd
from pymongo import MongoClient


client=MongoClient('mongodb://root:' + '5768116' + '@121.196.220.14')
db = client.cundan
collection = db.cundan





shijian=time.strftime('%Y-%m-%d',time.localtime(time.time()))
shijian=str(shijian)
shijian = datetime.datetime.strptime(shijian, "%Y-%m-%d")
shijian1=shijian-datetime.timedelta(days=20)




url='yinhang.csv'
df = pd.read_table(url,sep=',',encoding='GB18030')   #用dictreader根据行内容查找


for i in range(0,len(df)):    
        cursor = collection.find({"$and":[{'发行日':{'$gte':str(shijian1)}},{'发行人':{'$regex':df.loc[i,'银行']}}]}) 
        cun_df = pd.DataFrame(list(cursor))
        if cun_df.empty:

            print('11')
        else:
            cun_df1=cun_df[cun_df['期限']=='1月']
            cun_df1=cun_df1.reset_index(drop=False)

            if cun_df1.empty:
               print(111)
            else:
                df.loc[i,'1月利率']=cun_df1.loc[(len(cun_df1)-1),'收益率']
                df.loc[i,'1月发行日']=cun_df1.loc[(len(cun_df1)-1),'发行日']
                df.loc[i,'1月发行量']=cun_df1.loc[(len(cun_df1)-1),'计划发行(亿)']

            cun_df2=cun_df[cun_df['期限']=='3月']
            cun_df2=cun_df2.reset_index(drop=False)

            if cun_df2.empty:
               print(111)
            else:
                print(cun_df2)
                df.loc[i,'3月利率']=cun_df2.loc[(len(cun_df2)-1),'收益率']
                df.loc[i,'3月发行日']=cun_df2.loc[(len(cun_df2)-1),'发行日']
                df.loc[i,'3月发行量']=cun_df2.loc[(len(cun_df2)-1),'计划发行(亿)']
                
            cun_df3=cun_df[cun_df['期限']=='6月']
            cun_df3=cun_df3.reset_index(drop=False)

            if cun_df3.empty:
               print(111)
            else:
                df.loc[i,'6月利率']=cun_df3.loc[(len(cun_df3)-1),'收益率']
                df.loc[i,'6月发行日']=cun_df3.loc[(len(cun_df3)-1),'发行日']
                df.loc[i,'6月发行量']=cun_df3.loc[(len(cun_df3)-1),'计划发行(亿)']

            cun_df4=cun_df[cun_df['期限']=='9月']
            cun_df4=cun_df4.reset_index(drop=False)

            if cun_df4.empty:
               print(111)
            else:
                df.loc[i,'9月利率']=cun_df4.loc[(len(cun_df4)-1),'收益率']
                df.loc[i,'9月发行日']=cun_df4.loc[(len(cun_df4)-1),'发行日']
                df.loc[i,'9月发行量']=cun_df4.loc[(len(cun_df4)-1),'计划发行(亿)']



            cun_df5=cun_df[cun_df['期限']=='1年']
            cun_df5=cun_df5.reset_index(drop=False)

            if cun_df5.empty:
               print(111)
            else:
                df.loc[i,'1年利率']=cun_df5.loc[(len(cun_df5)-1),'收益率']
                df.loc[i,'1年发行日']=cun_df5.loc[(len(cun_df5)-1),'发行日']
                df.loc[i,'1年发行量']=cun_df5.loc[(len(cun_df5)-1),'计划发行(亿)']



#cursor = collection.find({ "$and":[{'发行日':'2018-03-06'}] })  

#df = pd.DataFrame(list(cursor))

#print(df)
df.to_csv('重要交易对手同业存单.csv',mode='w',header=True)


11
11
11
11
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        1  5d231c6d12e3441414c1786b      0  111904043  19中国银行CD043  报价发行   
1        3  5d27207012e3440df06d1af8      1  111904045  19中国银行CD045  报价发行   

          发行日 实际发行(亿) 息票类型   收益率  期限        爬取日期 票面利率  计划发行(亿)   评级  银行分类  
0  2019-07-09    None   零息  2.42  3月  2019-07-08   --      0.9  AAA  中国银行  
1  2019-07-12    None   零息  2.43  3月  2019-07-11   --      2.4  AAA  中国银行  
111
111
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        1  5c45928fe5b12d1ee0a53553    102  111906015  19交通银行CD015  报价发行   
1        7  5d1b374812e34418c821c02c      2  111906178  19交通银行CD178  报价发行   
2       10  5d1c86d412e3440740d99de0      1  111906181  19交通银行CD181  报价发行   
3       12  5d1dd6f912e3441174b1c7f1      3  111906183  19交通银行CD183  报价发行   
4       15  5d1f35ba12e34416e49b29a2      2  111906186  19交通银行CD186  报价发行   
5       17  5d231c6d12e3441414c1786e      3  11190

11
111
111
111
111
11
111
111
111
111
11
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        1  5d1c86d412e3440740d99e15     54  111981778  19成都银行CD144  报价发行   
1        2  5d1dd6f912e3441174b1c827     57  111981895  19成都银行CD145  报价发行   
2        4  5d231c6d12e3441414c178b5     74  111982126  19成都银行CD147  报价发行   
3        6  5d246f4f12e3441744b7a397     53  111982202  19成都银行CD149  报价发行   
4        8  5d25ca6b12e3441124ca7324     47  111982294  19成都银行CD151  报价发行   
5        9  5d27207012e3440df06d1b38     65  111982406  19成都银行CD152  报价发行   

          发行日 实际发行(亿) 息票类型     收益率  期限        爬取日期 票面利率  计划发行(亿)  评级 银行分类  
0  2019-07-04    None   零息  2.5998  3月  2019-07-03   --      4.8  --   城商  
1  2019-07-05    None   零息  2.6000  3月  2019-07-04   --      2.3  --   城商  
2  2019-07-09    None   零息  2.6999  3月  2019-07-08   --      3.7  --   城商  
3  2019-07-10    None   零息  2.7302  3月  2019-07-09   --     10.0  --   城商  
4  2019-07-11    None   零息  2.7302 

111
111
111
111
111
11
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        0  5d0cf10412e3441600ad51d2    103  111981040  19桂林银行CD122  报价发行   
1        1  5d0cf10412e3441600ad51d3    104  111981041  19桂林银行CD123  报价发行   
2        3  5d11fbe012e3441ad822a32a     70  111981261  19桂林银行CD125  报价发行   
3        4  5d27207012e3440df06d1b83    140  111982481  19桂林银行CD126  报价发行   

          发行日 实际发行(亿) 息票类型  收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-06-24    None   零息  3.2  3月  2019-06-21   --      2.0  AA+   城商  
1  2019-06-24    None   零息  3.2  3月  2019-06-21   --      2.0  AA+   城商  
2  2019-06-26    None   零息  3.2  3月  2019-06-25   --      2.0  AA+   城商  
3  2019-07-12    None   零息  3.0  3月  2019-07-11   --      2.8  AA+   城商  
   level_0                       _id  index       债券代码           发行人  发行方式  \
0        1  5d0cf10412e3441600ad51a8     61  111980998  19哈尔滨银行CD041  报价发行   
1        3  5d10c0a212e3441d646c655d     84  111981159  19哈尔滨

111
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        2  5d19ecfa12e3440aac7d05e7     82  111981566  19宁波银行CD118  报价发行   
1        3  5d1b374812e34418c821c0ad    131  111981711  19宁波银行CD119  报价发行   
2        4  5d1c86d412e3440740d99e44    101  111981826  19宁波银行CD120  报价发行   
3        6  5d1dd6f912e3441174b1c843     85  111981923  19宁波银行CD122  报价发行   
4        9  5d1f35ba12e34416e49b29e8     72  111982043  19宁波银行CD125  报价发行   
5       11  5d231c6d12e3441414c178b1     70  111982122  19宁波银行CD127  报价发行   
6       13  5d246f4f12e3441744b7a3af     77  111982226  19宁波银行CD129  报价发行   
7       15  5d25ca6b12e3441124ca735c    103  111982350  19宁波银行CD131  报价发行   
8       17  5d27207012e3440df06d1b5d    102  111982443  19宁波银行CD133  报价发行   

          发行日 实际发行(亿) 息票类型   收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-07-02    None   零息  2.55  3月  2019-07-01   --      5.0  AAA   城商  
1  2019-07-03    None   零息  2.55  3月  2019-07-02   --      5.0  AAA   城

11
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        0  5d231c6d12e3441414c178da    111  111982164  19台州银行CD023  报价发行   

          发行日 实际发行(亿) 息票类型   收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-07-09    None   零息  3.15  3月  2019-07-08   --     18.0  AA+   城商  
111
111
   level_0                       _id  index       债券代码          发行人  发行方式  \
0        1  5d0cf10412e3441600ad51d9    110  111981047  19天津银行CD136  报价发行   
1        2  5d10c0a212e3441d646c6555     76  111981151  19天津银行CD137  报价发行   
2        5  5d11fbe012e3441ad822a336     82  111981273  19天津银行CD140  报价发行   
3        6  5d134add12e3441860b2401a     53  111981375  19天津银行CD141  报价发行   

          发行日 实际发行(亿) 息票类型   收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-06-24    None   零息  2.90  3月  2019-06-21   --     15.1  AAA   城商  
1  2019-06-25    None   零息  2.70  3月  2019-06-24   --      2.5  AAA   城商  
2  2019-06-26    None   零息  2.65  3月  2019-06-25   --      0.8  AAA  

111
111
111
11
111
111
111
111
111
111
111
111
111
   level_0                       _id  index       债券代码             发行人  发行方式  \
0        9  5d1c86d412e3440740d99e49    106  111981831  19广东顺德农商行CD084  报价发行   
1       10  5d1dd6f912e3441174b1c862    116  111981954  19广东顺德农商行CD085  报价发行   
2       14  5d1f35ba12e34416e49b2a03     99  111982070  19广东顺德农商行CD089  报价发行   

          发行日 实际发行(亿) 息票类型  收益率  期限        爬取日期 票面利率  计划发行(亿)   评级 银行分类  
0  2019-07-04    None   零息  2.6  3月  2019-07-03   --      1.0  AAA  农商行  
1  2019-07-05    None   零息  2.6  3月  2019-07-04   --      2.5  AAA  农商行  
2  2019-07-08    None   零息  2.6  3月  2019-07-05   --      1.4  AAA  农商行  
   level_0                       _id  index       债券代码              发行人  发行方式  \
0        0  5d0cf10412e3441600ad51e4    121  111981058  19广州农村商业银行CD075  报价发行   
1        5  5d10c0a212e3441d646c6567     94  111981169  19广州农村商业银行CD080  报价发行   
2        8  5d11fbe012e3441ad822a33d     89  111981280  19广州农村商业银行CD083  报价发行   

       

11
11
11
11


#  微信群价格

In [2]:
# -*- coding: cp936 -*-
import pandas as pd

import matplotlib.pyplot as plt
#from PIL import Image
import numpy as np

import jieba,os
import jieba.analyse
import json,csv,re,time,datetime
from pymongo import MongoClient
#from wordcloud import WordCloud
shijian=time.strftime('%Y-%m-%d',time.localtime(time.time()))

shijian = datetime.datetime.strptime(shijian, "%Y-%m-%d")
shijian=str(shijian.strftime("%Y-%m-%d"))

print(shijian)
shuru=input('请输入要统计的日期,格式为“2017-11-01”，不输入自定义为今天:')
if shuru!='':
    shijian=shuru

client=MongoClient('mongodb://root:' + '5768116' + '@121.196.220.14')
db = client.piaofen
collection = db.piaofen
cursor = collection.find({"$and":[{'time':shijian},{'hanglei2':1}]})
df = pd.DataFrame(list(cursor))

df = df.sort_values(by='time2', ascending=True)
df=df.drop_duplicates(['content'],keep='last')   #删除连接中的重复行

for i in range (1,len(df)):
      zhaop= re.search('[^0-9][0-9]\.[0-9]+[^元万亿]',str(df.loc[i,'content']))
      if zhaop:
         df.loc[i,'jiage']=1
      zhaop1= re.search('[0-9]+%',str(df.loc[i,'content']))
      if zhaop1:
         df.loc[i,'jiage']=1

for i in range (1,len(df)):   
      zhaop3= re.search('[^0-9][1-9][0-9][0-9][^0-9万天]',str(df.loc[i,'content']))
      if zhaop3:
             df.loc[i,'jiage']=1
      

df=df[df['jiage']==1]
df1=df[(df['shou']+df['chu']+df['shoudai']+df['chudai']+df['shouhui']+df['chuhui'])>0]
df1=df1.sort_values('hanglei1', ascending=True)  #吧银行类别排序

columns = ['time','time2','hanglei1','nickname','content']
df1.to_csv('票据价格.csv',mode='w',header=True,columns=columns)

df2=df[(df['shoufu']+df['chufu'])>0]
df2=df2.sort_values('hanglei1', ascending=True)  #吧银行类别排序
columns = ['time','time2','hanglei1','nickname','content']
df2.to_csv('福费廷价格.csv',mode='w',header=True,columns=columns)

df3=df[(df['shoucun']+df['chucun'])>0]
df3=df3.sort_values('hanglei1', ascending=True)  #吧银行类别排序
columns = ['time','time2','hanglei1','nickname','content']
df3.to_csv('资金价格.csv',mode='w',header=True,columns=columns)

df4=df[(df['shouli']+df['chuli'])>0]
df4=df4.sort_values('hanglei1', ascending=True)  #吧银行类别排序
columns = ['time','time2','hanglei1','nickname','content']
df4.to_csv('理财价格.csv',mode='w',header=True,columns=columns)


2019-06-27
请输入要统计的日期,格式为“2017-11-01”，不输入自定义为今天:
